# Stars: 

### Day 11: Cosmic Expansion

#### Part 1

In [36]:
# read all lines in text file
with open("test1_input.txt", "r") as file:
    lines = file.readlines() 

# remove newline text from each line
lines = [line.rstrip("\n").split(" ") for line in lines]

# get metadata
n_rows = len(lines)
n_cols = len(lines[0][0])
print(f"Shape: {n_cols} columns, {n_rows} rows")

Shape: 10 columns, 10 rows


In [37]:
def display_grid(grid):
    for row in grid:
        row = [str(val) for val in row]
        print("".join(row))

def generate_grid(lines, n_rows, n_cols, print_grid=False):

    grid = [[' ' for _ in range(n_rows)] for _ in range(n_cols)]

    for i, row in enumerate(lines):
        for j, element in enumerate(row[0]):
            grid[i][j] = element

    if print_grid==True:
        display_grid(grid)
    
    return grid

In [38]:
def find_empty_rows_and_columns(grid):
    empty_rows = []
    empty_columns = []

    # Check empty rows
    for i, row in enumerate(grid):
        if all(cell == "." for cell in row):
            empty_rows.append(i)

    # Check empty columns
    num_columns = len(grid[0])
    for j in range(num_columns):
        if all(grid[i][j] == "." for i in range(len(grid))):
            empty_columns.append(j)

    return empty_rows, empty_columns

def insert_new_row(grid, row_index, new_row):
    grid.insert(row_index, new_row)
    return grid

def insert_new_column(grid, column_index, new_column):
    for row in grid:
        row.insert(column_index, new_column)
    return grid

In [39]:
def expand_grid(grid, loops=1, print_grid=False):

    for loop in range(loops):

        empty_rows, empty_columns = find_empty_rows_and_columns(grid)

        for row_index in empty_rows[::-1]:
            new_row = grid[row_index]
            grid = insert_new_row(grid, row_index, new_row)

        for col_index in empty_columns[::-1]:
            for i, row in enumerate(grid):
                new_row = row[:]
                new_row[col_index:col_index] = ["."]
                grid[i] = new_row
    
    if print_grid==True:
        print("\nExpanded grid:")
        display_grid(grid)

    return grid

In [40]:
def enumerate_galaxies(grid, print_grid=False):
    i = 1
    galaxies = {}
    for row_index, row in enumerate(grid):
        for col_index, col in enumerate(row):
            val = grid[row_index][col_index]
            if val == "#":
                galaxy = str(i)
                grid[row_index][col_index] = galaxy
                galaxies[galaxy] = [row_index, col_index]
                i += 1
    
    if print_grid==True:
        print("\nEnumerated grid:")
        display_grid(grid)

    return galaxies, grid

In [41]:
def find_obstacles(grid, print_grid=False):
    n_rows = len(grid)
    n_cols = len(grid[0])
    obs_grid = [[' ' for _ in range(n_cols)] for _ in range(n_rows)]
    for row_index, row in enumerate(obs_grid):
        for col_index, col in enumerate(row):
            val = grid[row_index][col_index]
            if val == ".":
                obs_grid[row_index][col_index] = 0
            else:
                obs_grid[row_index][col_index] = 1

    if print_grid==True:
        print("\nObstacles:")
        display_grid(obs_grid)

    return obs_grid

In [42]:
def pair_galaxies(galaxies):
    pairs = []
    for galaxy1 in galaxies:
        for galaxy2 in galaxies:
            if galaxy1 != galaxy2:
                pair = sorted([galaxy1, galaxy2])
                if pair not in pairs:
                    pairs.append(pair)
    return pairs

In [43]:
grid = generate_grid(lines, n_rows, n_cols, print_grid=True)

# cosmic expansion - find and duplicate empty rows and columns
grid = expand_grid(grid, loops=1, print_grid=True)

# enumerate galaxies
galaxies, grid = enumerate_galaxies(grid, print_grid=True)

# find obstacles
obstacles = find_obstacles(grid, print_grid=True)

# pair up galaxies
pairs = pair_galaxies(galaxies.keys())
print(f"\nThere are {len(pairs)} galaxy pairs")

...#......
.......#..
#..#......
..........
......#...
.#........
.........#
..........
.......#..
#...#.....

Expanded grid:
....#........
.........#...
#...#........
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......

Enumerated grid:
....1........
.........2...
3...4........
.............
.............
........5....
.6...........
............7
.............
.............
.........8...
9....10.......

Obstacles:
0000100000000
0000000001000
1000100000000
0000000000000
0000000000000
0000000010000
0100000000000
0000000000001
0000000000000
0000000000000
0000000001000
1000010000000

There are 45 galaxy pairs


In [44]:
def check_direction(coord1, coord2):

    i1, j1 = coord1
    i2, j2 = coord2

    if i1 == i2:
        row_dir = ""
    elif i1 < i2:
        row_dir = "D"
    elif i1 > i2:
        row_dir = "U"

    if j1 == j2:
        col_dir = ""
    elif j1 < j2:
        col_dir = "R"
    elif j1 > j2:
        col_dir = "L"

    dir = [row_dir, col_dir]
    
    return dir

In [45]:
dir_offset_map = {
    "D": ( 1, 0),
    "U": (-1, 0),
    "L": ( 0,-1),
    "R": ( 0, 1),
    "" : ( 0, 0)
}

In [46]:
def find_next_pos(last_cell, current_path, offset):

    new_i = last_cell[0] + offset[0]
    new_j = last_cell[1] + offset[1]
    new_cell = [new_i, new_j]
        
    current_path_copy = current_path.copy()
    current_path_copy.append(new_cell)

    return new_cell, current_path_copy

In [51]:
def find_path(path=[[0, 4]], end=[1, 9], dir=['D', 'R'], grid=grid):
    
    row_dir, col_dir = dir
    row_offset = dir_offset_map[row_dir]
    col_offset = dir_offset_map[col_dir]

    last_cell = path[-1]

    # iterate through rows first until last_cell i matches destination i
    if last_cell[0] != end[0]:

        while True:

            next_cell, path = find_next_pos(last_cell, path, row_offset)
            next_cell_val = grid[next_cell[0]][next_cell[1]]
            print(next_cell_val)
            last_cell = next_cell

            if end[0] == next_cell[0]:
                break

    # iterate through columns until last_cell j matches destination j
    if last_cell[1] != end[1]:

        while True:

            next_cell, path = find_next_pos(last_cell, path, col_offset)
            next_cell_val = grid[next_cell[0]][next_cell[1]]
            print(next_cell_val)
            last_cell = next_cell

            if end[1] == next_cell[1]:
                break

    return path

In [52]:
pair_paths = {}
total_lengths = 0
for pair in pairs:
    print(pair)
    init_galaxy = galaxies[pair[0]]
    dest_galaxy = galaxies[pair[1]]
    dir = check_direction(init_galaxy, dest_galaxy)
    path = find_path([init_galaxy], dest_galaxy, dir, grid)
    join_pair = " to ".join(pair)
    pair_paths[join_pair] = path
    path_len = len(path) - 1
    total_lengths += path_len

# for key, val in pair_paths.items():
#     print(key, "\t", len(val)-1, "steps")

print("The sum of the shortest path between all galaxies =", total_lengths)

['1', '2']
.
.
.
.
.
2
['1', '3']
.
4
.
.
.
3
['1', '4']
.
4
['1', '5']
.
4
.
.
.
.
.
.
5
['1', '6']
.
4
.
.
.
.
.
.
6
['1', '7']
.
4
.
.
.
.
.
.
.
.
.
.
.
.
7
['1', '8']
.
4
.
.
.
.
.
.
.
.
.
.
.
.
8
['1', '9']
.
4
.
.
.
.
.
.
.
.
.
.
.
.
9
['1', '10']
.
4
.
.
.
.
.
.
.
.
.
10
['2', '3']
.
.
.
.
.
4
.
.
.
3
['2', '4']
.
.
.
.
.
4
['2', '5']
.
.
.
.
5
['2', '6']
.
.
.
.
.
.
.
.
.
.
.
.
6
['2', '7']
.
.
.
.
.
.
.
.
7
['2', '8']
.
.
.
.
.
.
.
.
8
['2', '9']
.
.
.
.
.
.
.
.
8
.
.
.
.
10
.
.
.
.
9
['10', '2']
.
.
.
.
.
.
.
.
.
.
.
.
.
2
['3', '4']
.
.
.
4
['3', '5']
.
.
.
.
.
.
.
.
.
.
5
['3', '6']
.
.
.
.
6
['3', '7']
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
7
['3', '8']
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
8
['3', '9']
.
.
.
.
.
.
.
.
9
['10', '3']
.
.
.
.
.
.
.
.
.
4
.
.
.
3
['4', '5']
.
.
.
.
.
.
5
['4', '6']
.
.
.
.
.
.
6
['4', '7']
.
.
.
.
.
.
.
.
.
.
.
.
7
['4', '8']
.
.
.
.
.
.
.
.
.
.
.
.
8
['4', '9']
.
.
.
.
.
.
.
.
.
.
.
.
9
['10', '4']
.
.
.
.
.
.
.
.
.
4
['5', '6']
.
.
.
.
.
.
.
6
['5', '7'

In [49]:
# def limit_grid(start, end, grid, print_grid=False):
#     n_rows = len(grid)
#     n_cols = len(grid[0])

#     min_row = min(start[0], end[0])
#     max_row = max(start[0], end[0])
#     min_col = min(start[1], end[1])
#     max_col = max(start[1], end[1])

#     i = 2
#     j = 3
#     if min_row-i < 0: min_row_range = 0
#     else: min_row_range = min_row-i
#     if min_col-i < 0: min_col_range = 0
#     else: min_col_range = min_col-i
#     if max_row+j > n_rows: max_row_range = n_rows
#     else: max_row_range = max_row+j
#     if max_col+j > n_cols: max_col_range = n_cols
#     else: max_col_range = max_col+j
    
#     row_range = [min_row_range, max_row_range]
#     col_range = [min_col_range, max_col_range]
#     # print("Rows:", row_range)
#     # print("Cols:", col_range)

#     grid_ltd = []
#     for row_index, row in enumerate(grid):
#         row_ltd = []
#         if row_index in range(row_range[0], row_range[1]):
#             for col_index, col in enumerate(row):
#                 if col_index in range(col_range[0], col_range[1]):
#                     val = grid[row_index][col_index]
#                     row_ltd.append(val)
#         if len(row_ltd)>0:
#             grid_ltd.append(row_ltd)

#     if print_grid == True:
#         print("Limited grid:")
#         display_grid(grid_ltd)
    
#     return grid_ltd

# # example
# start = [0, 4] # galaxy 1
# end = [5, 8]   # galaxy 4
# grid_ltd = limit_grid(start, end, grid, print_grid=True)

In [50]:
def find_paths_recursive(current_path=[(0, 4)], end=(5, 8), grid=grid_ltd, solutions=[]):
    
    n_rows = len(grid)
    n_cols = len(grid[0])

    last_cell = current_path[-1]

    offsets = [(0,1), (1,0), (0,-1), (-1,0)]

    for offset in offsets:

        x, y = offset
        new_i = last_cell[0] + x
        new_j = last_cell[1] + y
        new_cell = (new_i, new_j)

        # Check if new cell is in grid
        if 0 > new_i or new_i >= n_rows or 0 > new_j or new_j >= n_cols:
            # print(f">>> {new_cell} out of index")
            continue

        # Check if new cell is already in path
        if new_cell in current_path[1:]:
            # print(f">>> {new_cell} already in path")
            continue

        current_path_copy = current_path.copy()
        current_path_copy.append(new_cell)

        # print(current_path_copy)

        if (new_i==n_rows-1 and new_j == n_cols-1) or (end in current_path_copy):
            solutions.append(current_path_copy)

        solutions.append(current_path_copy)

        # Create new current_path array for every direction
        find_paths_recursive(current_path_copy, end, grid, solutions)
        break

    for solution in solutions:
        if solution[-1] == end:
            print("Found")
            return solution

# example
start = (6, 2) # galaxy 5
end = (11, 5)   # galaxy 9
grid_ltd = limit_grid(start, end, grid, print_grid=True)
solution = find_paths_recursive(current_path=[start], end=end, grid=grid_ltd, solutions=[])
solution

NameError: name 'grid_ltd' is not defined

In [ ]:
def is_valid_move(grid, row, col):
    # Check if the move is within the grid boundaries and not an obstacle
    return 0 <= row < len(grid) and 0 <= col < len(grid[0]) and grid[row][col] != '.'

def shortest_path(grid, start, end):
    # Define possible moves: up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    queue = [(start, 0)]  # Initialize queue with start point and distance
    visited = set()  # Set to store visited points

    while queue:
        current_point, distance = queue.pop(0)
        if current_point == end:
            return distance  # Return distance when end point is reached
        visited.add(current_point)
        for dx, dy in directions:
            new_row, new_col = current_point[0] + dx, current_point[1] + dy
            if is_valid_move(grid, new_row, new_col) and (new_row, new_col) not in visited:
                queue.append(((new_row, new_col), distance + 1))

    return -1  # Return -1 if no path is found

In [ ]:
# Finding the shortest path
# start = (6, 2) # galaxy 5
# end = (11, 5)   # galaxy 9
start = (6, 1)  # Starting point
end = (10, 5)   # Ending point
shortest_distance = shortest_path(grid, start, end)
print("Shortest distance between point 5 and point 9:", shortest_distance)

In [ ]:
import heapq

In [ ]:
def heuristic(a, b):
    """
    Calculate the Manhattan distance heuristic between points a and b.
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


def astar(grid, start_pos, goal):
    """
    Find the shortest path from start to goal on the grid using A* algorithm.
    """
    # Define possible movement directions: up, down, left, right
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # Initialize lists for open and closed nodes
    open_list = []
    closed_set = set()

    # Add the start node to the open list
    heapq.heappush(open_list, (0, start_pos))
    
    # Initialize dictionaries to keep track of parents and costs
    parent = {}
    g_score = {start_pos: 0}
    
    while open_list:
        # Pop the node with the lowest f-score from the open list
        current_cost, current_node = heapq.heappop(open_list)
        
        # Check if we have reached the goal
        if current_node == goal:

            # Reconstruct the path
            path = []
            while current_node in parent:
                path.append(current_node)
                current_node = parent[current_node]
            path.append(start_pos)
            path.reverse()
            return path
        
        # Add current node to closed set
        closed_set.add(current_node)
        
        # Explore neighbors
        for dx, dy in directions:
            neighbor = current_node[0] + dx, current_node[1] + dy

            # Check if neighbor is within grid bounds
            if 0 <= neighbor[0] < len(grid) and 0 <= neighbor[1] < len(grid[0]):
                
                # Check if neighbor is not an obstacle
                if grid[neighbor[0]][neighbor[1]] == '.':
                    
                    # Calculate tentative g score
                    tentative_g_score = g_score[current_node] + 1
                    
                    # Check if neighbor is in closed set or has a better g score
                    if neighbor not in closed_set or tentative_g_score < g_score.get(neighbor, float('inf')):
                        
                        # Update parent and g score
                        parent[neighbor] = current_node
                        g_score[neighbor] = tentative_g_score
                        
                        # Calculate f score and add neighbor to open list
                        f_score = tentative_g_score + heuristic(neighbor, goal)
                        heapq.heappush(open_list, (f_score, neighbor))
    
    # If no path found, return empty list
    return []


def find_shortest_paths(grid, start_num):
    """
    Find the shortest paths from 1 to every other number in the grid.
    """
    # Find the coordinates of all numbers in the grid
    numbers = {}
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j].isdigit():
                numbers[int(grid[i][j])] = (i, j)
    
    # Find shortest paths from start number to every other number
    shortest_paths = {}
    for num in numbers:
        if num != start_num:
            shortest_path = astar(grid, numbers[start_num], numbers[num])
            shortest_paths[num] = shortest_path
    
    return shortest_paths


In [ ]:
grid

In [ ]:
# Find the shortest paths from 1 to every other number
shortest_paths = find_shortest_paths(grid, 1)
display(shortest_paths)
for num, path in shortest_paths.items():
    print(f"Shortest path from 1 to {num}: {path}")

In [ ]:
shortest_paths